# NB3 Rating Predictions
- This is work in progress, need to play more with data.
- But it conveys the gist.
- Need to work on describing interpretability and further optimizing

In [1]:
import numpy as np
import pandas as pd

In [2]:
reviews_features = pd.read_csv("reviews_features.csv")
print(reviews_features.shape)
reviews_features.head()
# Remove rows with missing values in the target variable 'Recommend Flag'
reviews_features = reviews_features.dropna(subset=['Recommend Flag'])
print(reviews_features.shape)
reviews_features = reviews_features.dropna(subset=['Rating'])
print(reviews_features.shape)

(23472, 18)
(23472, 18)
(23472, 18)


In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

# Select columns for each transformation type
predicts = ['Rating', 'Recommend Flag']
labels = ['Product ID']
categorical_cols = ['Category', 'Subcategory1', 'SubCategory2', 'Location', 'Channel', 'age_bins', ]
numeric_cols = ['review_len', 'noun_strength', 'polarity', 'subjectivity_score', 'compound_VADER_score', 'unique_word_fraction'] 
binary_cols = ['is_gibberish']

# One-Hot Encoding for categorical features
ohe = OneHotEncoder(sparse_output=False, drop='first') 
categorical_encoded = ohe.fit_transform(reviews_features[categorical_cols])

# Create DataFrame with OHE features
ohe_df = pd.DataFrame(categorical_encoded, columns=ohe.get_feature_names_out(categorical_cols))

# Scaling numerical features
scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(reviews_features[numeric_cols])

# Create DataFrame with scaled numerical features
numeric_df = pd.DataFrame(numeric_scaled, columns=numeric_cols)

# Combine all features into a final DataFrame
reviewsFE_final_df = pd.concat([ohe_df, numeric_df, reviews_features[binary_cols].reset_index(drop=True)], axis=1)


reviewsFE_final_df = pd.concat([reviewsFE_final_df, reviews_features[['Product ID','Rating', 'Recommend Flag']]], axis=1)

to_drop = ['Product ID','Rating', 'Recommend Flag']

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=50, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(reviews_features['lemmatized'])
words_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
words_df

,_x000d_,beautiful,big,bit,buy,color,comfortable,cute,dress,fabric,...,skirt,small,soft,sweater,think,try,waist,want,wear,work
0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.000000,0.0,0.221448,0.000000,0.000000,0.000000,0.168004,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.228564,0.000000,0.168755,0.000000,...,0.0,0.610343,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.239856
3,0.0,0.0,0.000000,0.0,0.398506,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.294867,0.000000
4,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.207795,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23467,0.0,0.0,0.000000,0.0,0.000000,0.308829,0.000000,0.000000,0.592446,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
23468,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.378256,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
23469,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.397934,0.000000,0.372533,...,0.0,0.000000,0.000000,0.0,0.0,0.433679,0.000000,0.000000,0.000000,0.437153
23470,0.0,0.0,0.240827,0.0,0.206671,0.000000,0.000000,0.405721,0.470379,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.244071,0.243254,0.000000,0.000000


In [5]:
reviewsFE_final_df = pd.concat([reviewsFE_final_df, words_df], axis=1)

# Reset index and show the final DataFrame
reviewsFE_final_df.reset_index(drop=True, inplace=True)

# Display the final DataFrame
print(reviewsFE_final_df.columns)

Index(['Category_General Petite', 'Category_Initmates', 'Subcategory1_Dresses',
       'Subcategory1_Intimate', 'Subcategory1_Jackets', 'Subcategory1_Tops',
       'Subcategory1_Trend', 'SubCategory2_Casual bottoms',
       'SubCategory2_Chemises', 'SubCategory2_Dresses',
       'SubCategory2_Fine gauge', 'SubCategory2_Intimates',
       'SubCategory2_Jackets', 'SubCategory2_Jeans', 'SubCategory2_Knits',
       'SubCategory2_Layering', 'SubCategory2_Legwear', 'SubCategory2_Lounge',
       'SubCategory2_Outerwear', 'SubCategory2_Pants', 'SubCategory2_Shorts',
       'SubCategory2_Skirts', 'SubCategory2_Sleep', 'SubCategory2_Sweaters',
       'SubCategory2_Swim', 'SubCategory2_Trend', 'Location_Chennai',
       'Location_Gurgaon', 'Location_Mumbai', 'Channel_Web', 'age_bins_35-44',
       'age_bins_45-54', 'age_bins_55-64', 'age_bins_above 65',
       'age_bins_upto 24', 'review_len', 'noun_strength', 'polarity',
       'subjectivity_score', 'compound_VADER_score', 'unique_word_fraction'

In [6]:
from sklearn.model_selection import train_test_split

X = reviewsFE_final_df.drop(columns=to_drop)
y = reviewsFE_final_df['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
lr_classifier = LogisticRegression(max_iter=5000)
lr_classifier.fit(X_train,y_train)
print("Training Accuracy\n", accuracy_score(y_train,lr_classifier.predict(X_train)))
print("Test Accuracy\n", accuracy_score(y_test,lr_classifier.predict(X_test)))

print('CLASSIFICATION REPORT: Logistic Regression')
print("Training\n", classification_report(y_train,lr_classifier.predict(X_train)))
print("Test \n", classification_report(y_test,lr_classifier.predict(X_test)))

Training Accuracy
 0.6052084997603451
Test Accuracy
 0.5891373801916933
CLASSIFICATION REPORT: Logistic Regression
Training
               precision    recall  f1-score   support

           1       0.43      0.20      0.27       666
           2       0.33      0.11      0.16      1242
           3       0.37      0.27      0.31      2266
           4       0.42      0.19      0.26      4062
           5       0.67      0.92      0.77     10541

    accuracy                           0.61     18777
   macro avg       0.44      0.34      0.36     18777
weighted avg       0.55      0.61      0.55     18777

Test 
               precision    recall  f1-score   support

           1       0.39      0.19      0.26       176
           2       0.38      0.14      0.20       323
           3       0.31      0.20      0.24       605
           4       0.38      0.17      0.23      1015
           5       0.66      0.93      0.77      2576

    accuracy                           0.59      4695

In [8]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train,y_train)

print("Training Accuracy\n", accuracy_score(y_train,dt_classifier.predict(X_train)))
print("Test Accuracy\n", accuracy_score(y_test,dt_classifier.predict(X_test)))

print('CLASSIFICATION REPORT: Decision Tree Classifier')
print("Training\n", classification_report(y_train,dt_classifier.predict(X_train)))
print("Test \n", classification_report(y_test,dt_classifier.predict(X_test)))

Training Accuracy
 0.9937689726793417
Test Accuracy
 0.4696485623003195
CLASSIFICATION REPORT: Decision Tree Classifier
Training
               precision    recall  f1-score   support

           1       0.99      1.00      0.99       666
           2       1.00      1.00      1.00      1242
           3       0.99      0.99      0.99      2266
           4       0.99      0.99      0.99      4062
           5       0.99      1.00      1.00     10541

    accuracy                           0.99     18777
   macro avg       0.99      0.99      0.99     18777
weighted avg       0.99      0.99      0.99     18777

Test 
               precision    recall  f1-score   support

           1       0.18      0.15      0.16       176
           2       0.15      0.14      0.14       323
           3       0.24      0.25      0.25       605
           4       0.25      0.25      0.25      1015
           5       0.66      0.67      0.66      2576

    accuracy                           0.47     

In [9]:
from sklearn.ensemble import RandomForestClassifier

rfc_classifier = RandomForestClassifier(n_estimators=50, n_jobs = -1, oob_score = True,random_state=42)
rfc_classifier.fit(X_train,y_train)

print("Training Accuracy\n", accuracy_score(y_train,rfc_classifier.predict(X_train)))
print("Test Accuracy\n", accuracy_score(y_test,rfc_classifier.predict(X_test)))

print('CLASSIFICATION REPORT: Random Forest Classifier')
print("Training\n", classification_report(y_train,rfc_classifier.predict(X_train)))
print("Test \n", classification_report(y_test,rfc_classifier.predict(X_test)))

Training Accuracy
 0.9937157160355754
Test Accuracy
 0.5716719914802982
CLASSIFICATION REPORT: Random Forest Classifier
Training
               precision    recall  f1-score   support

           1       1.00      0.99      0.99       666
           2       1.00      0.99      1.00      1242
           3       1.00      0.99      0.99      2266
           4       0.99      0.99      0.99      4062
           5       0.99      1.00      0.99     10541

    accuracy                           0.99     18777
   macro avg       1.00      0.99      0.99     18777
weighted avg       0.99      0.99      0.99     18777

Test 
               precision    recall  f1-score   support

           1       0.44      0.14      0.21       176
           2       0.32      0.11      0.17       323
           3       0.32      0.16      0.21       605
           4       0.28      0.12      0.16      1015
           5       0.63      0.94      0.76      2576

    accuracy                           0.57     